In [2]:
import json
import re
def removeCCppComment(text):  # https://stackoverflow.com/a/18234680
    # Return a string containing only the newline chars contained in strIn
    def blotOutNonNewlines(strIn):
        return "" + ("\n" * strIn.count('\n'))

    def replacer(match):
        s = match.group(0)
        # Matched string is //...EOL or /*...*/  ==> Blot out all non-newline chars
        if s.startswith('/'):
            return blotOutNonNewlines(s)
        else:  # Matched string is '...' or "..."  ==> Keep unchanged
            return s

    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )

    return re.sub(pattern, replacer, text)

# D = json.loads(J) # NOTE, documentation string might throw exeception
def D2C(J, sub_ANY=True):
    if sub_ANY:
        J = re.sub(r'ANY\((.*)?\)', r'\1', J)
    D = json.loads(J)
    layout = D["layout"]
    layers = D["layers"]
    s = "const uint16_t PROGMEM keymaps[][MATRIX_ROWS][MATRIX_COLS] = {"
    for i in range(len(layers)):
        s += "\n  [%d] = %s(" % (i, layout)
        s += ", ".join(layers[i])
        s += ")"
        if i < len(layers)-1:
            s += ","
    s += "\n};"
    return s

def C2D(C, kb, km, layout):
    matches = re.findall(r"\[(\d+)\] = ([^\(\)]+)\((.*)\)",C)
    D = {}
    D["version"] = 1
    D["notes"] = ""
    D["keyboard"] = kb # "ergodash/rev1"
    D["keymap"] = km # "ergodash_rev1_layout_4key_2u_inner_mine"
    D["layout"] = layout # "LAYOUT_4key_2u_inner"
    D["layers"] = []
    for match in matches:
        D["layers"].append( [kc.strip() for kc in re.split(r",(?![^\(]*\))", match[2])] ) # seems to work
        # D["layers"].append([kc.strip() for kc in match[2].split(", ")])
    D["author"] = ""
    return json.dumps(D)

def find_block(ccode, start, end):
    txt = removeCCppComment(ccode)
    lines = txt.split("\n")
    out = []
    appending = False
    for line in lines:
        line = line.strip()
        if len(line) == 0:
            continue
        if not appending:
            if start in line:
                out.append(start + line.split(start)[1]) # assume start occures at most once in the line..
                assert(len(line.split(start)) == 2) # otherwise could do start.join(line.split(start)[1:])
                appending = True
                continue # already appended line with special split
        if appending:
            if end in line: # includes ending, and stops after finding first occurence of start...end
                out.append(line.split(end)[0] + end)
                break
            else:
                out.append(line)
    return "\n".join(out)

def convert_json2c(cfile, kb, km, layout, jsonfile):
    with open(cfile, "r") as rfile:
        ccode = rfile.read()
        # print(ccode)
        tst = removeCCppComment(ccode)
        tst = find_block(ccode, start="const uint16_t PROGMEM keymaps", end=";")
        tst = C2D(tst, kb, km, layout)
        with open(jsonfile, "w") as wfile:
            wfile.write(tst)

In [ ]:
# convert_json2c("D:/QMK/qmk_firmware/keyboards/ergodash/rev1/keymaps/qmk_ergodash_keymap/keymap.c",
#                kb="ergodash/rev1",
#                km="ergodash_rev1_layout_4key_2u_inner_mine",
#                layout="LAYOUT_4key_2u_inner",
#                jsonfile="D:/QMK/qmk_firmware/keyboards/ergodash/rev1/keymaps/qmk_ergodash_keymap/converted.json"
#               )

In [32]:
print(D2C("""{"version":1,"notes":"","documentation":"...","keyboard":"ergodash/rev1","keymap":"ergodash_rev1_layout_4key_2u_inner_mine","layout":"LAYOUT_4key_2u_inner","layers":[["KC_ESC","...","KC_RGHT"],["KC_TRNS","...","KC_NO"]],"author":""}
"""))

const const uint16_t PROGMEM keymaps[][MATRIX_ROWS][MATRIX_COLS] = {
  [0] = LAYOUT_4key_2u_inner(KC_ESC, ..., KC_RGHT),
  [1] = LAYOUT_4key_2u_inner(KC_TRNS, ..., KC_NO)
};


In [28]:
print(C2D("""const uint16_t PROGMEM keymaps[][MATRIX_ROWS][MATRIX_COLS] = {
  [0] = LAYOUT_4key_2u_inner(KC_ESC, ..., KC_RGHT),
  [1] = LAYOUT_4key_2u_inner(KC_TRNS, ..., KC_TRNS),"""))

{"version": 1, "notes": "", "keyboard": "ergodash/rev1", "keymap": "ergodash_rev1_layout_4key_2u_inner_mine", "layout": "LAYOUT_4key_2u_inner", "layers": [["KC_ESC", "...", "KC_RGHT"], ["KC_TRNS", "...", "KC_TRNS"]], "author": ""}


In [3]:
in_str = """const uint16_t PROGMEM keymaps[][MATRIX_ROWS][MATRIX_COLS] = {
  [0] = LAYOUT_4key_2u_inner(KC_ESC, KC_1, KC_2, KC_3, KC_4, KC_5, KC_PSCR, KC_BSPC, KC_6, KC_7, KC_8, KC_9, KC_0, KC_DEL, KC_GRV, KC_Q, KC_W, KC_E, KC_R, KC_T, KC_MINS, KC_EQL, KC_Y, KC_U, KC_I, KC_O, KC_P, KC_BSLS, KC_TAB, KC_A, KC_S, KC_D, KC_F, KC_G, KC_LBRC, KC_RBRC, KC_H, KC_J, KC_K, KC_L, KC_SCLN, KC_QUOT, KC_LSFT, KC_Z, KC_X, KC_C, KC_V, KC_B, KC_LT, KC_UALT, KC_N, KC_M, KC_COMM, KC_DOT, KC_SLSH, KC_RSFT, KC_LCTL, KC_LGUI, KC_LALT, KC_LALT, MO(1), LT(2,KC_SPC), KC_SPC, MO(1), LT(3,KC_ENT), MO(5), KC_LEFT, KC_DOWN, KC_UP, KC_RGHT),
  [1] = LAYOUT_4key_2u_inner(KC_TRNS, KC_F1, KC_F2, KC_F3, KC_F4, KC_F5, KC_NO, KC_TRNS, KC_F6, KC_F7, KC_F8, KC_F9, KC_F10, KC_F11, KC_NO, KC_EXLM, KC_AT, KC_HASH, KC_DLR, KC_PERC, KC_NO, KC_NO, KC_CIRC, KC_AMPR, KC_ASTR, KC_LPRN, KC_RPRN, KC_F12, KC_TRNS, KC_1, KC_2, KC_3, KC_4, KC_5, KC_NO, KC_NO, KC_6, KC_7, KC_8, KC_9, KC_0, KC_NO, KC_TRNS, KC_Y, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_RALT, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_ENT, MO(4), KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS),
  [2] = LAYOUT_4key_2u_inner(KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_BSPC, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_DEL, KC_TILD, KC_EXLM, KC_CIRC, KC_AMPR, KC_LBRC, KC_UNDS, KC_NO, KC_NO, KC_NO, KC_RBRC, KC_BSLS, KC_NO, KC_NO, KC_NO, KC_TRNS, KC_MINS, KC_PLUS, KC_EQL, KC_LPRN, KC_LT, KC_SPC, KC_NO, KC_GT, KC_RPRN, KC_COLN, KC_QUES, KC_DQUO, KC_QUOT, KC_TRNS, KC_SLSH, KC_ASTR, KC_PIPE, KC_LCBR, KC_PERC, KC_NO, KC_NO, KC_HASH, KC_RCBR, KC_AT, KC_DLR, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_NO, KC_TRNS, KC_NO, KC_NO, KC_TRNS, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS),
  [3] = LAYOUT_4key_2u_inner(KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_BSPC, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_DEL, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_PGUP, KC_HOME, KC_UP, KC_END, KC_INS, KC_NO, KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_ENT, KC_PGDN, KC_LEFT, KC_DOWN, KC_RGHT, KC_DEL, KC_NO, KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_WH_U, KC_NO, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_WH_L, KC_WH_D, KC_WH_R, KC_NO),
  [4] = LAYOUT_4key_2u_inner(KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, RESET, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, UNICODE_MODE_FORWARD, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO),
  [5] = LAYOUT_4key_2u_inner(KC_NO, KC_F1, KC_F2, KC_F3, KC_F4, KC_F5, KC_NO, KC_BSPC, KC_F6, KC_F7, KC_F8, KC_F9, KC_F10, KC_DEL, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NLCK, KC_P7, KC_P8, KC_P9, KC_PMNS, KC_F12, KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_P4, KC_P5, KC_P6, KC_PPLS, KC_NO, KC_TRNS, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_NO, KC_TRNS, KC_NO, KC_P1, KC_P2, KC_P3, KC_NO, KC_NO, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, KC_TRNS, MO(4), MO(4), KC_TRNS, KC_TRNS, KC_P0, KC_PDOT, KC_COMM, KC_NO)
};"""
out_str = C2D(in_str)
print(out_str)
import datetime
t = datetime.datetime.now()
out_file = "ergodash_" + t.strftime('%Y%m%d') + ".json"
with open(out_file,'w') as thefile:
    thefile.write(out_str)


{"version": 1, "notes": "", "keyboard": "ergodash/rev1", "keymap": "ergodash_rev1_layout_4key_2u_inner_mine", "layout": "LAYOUT_4key_2u_inner", "layers": [["KC_ESC", "KC_1", "KC_2", "KC_3", "KC_4", "KC_5", "KC_PSCR", "KC_BSPC", "KC_6", "KC_7", "KC_8", "KC_9", "KC_0", "KC_DEL", "KC_GRV", "KC_Q", "KC_W", "KC_E", "KC_R", "KC_T", "KC_MINS", "KC_EQL", "KC_Y", "KC_U", "KC_I", "KC_O", "KC_P", "KC_BSLS", "KC_TAB", "KC_A", "KC_S", "KC_D", "KC_F", "KC_G", "KC_LBRC", "KC_RBRC", "KC_H", "KC_J", "KC_K", "KC_L", "KC_SCLN", "KC_QUOT", "KC_LSFT", "KC_Z", "KC_X", "KC_C", "KC_V", "KC_B", "KC_LT", "KC_UALT", "KC_N", "KC_M", "KC_COMM", "KC_DOT", "KC_SLSH", "KC_RSFT", "KC_LCTL", "KC_LGUI", "KC_LALT", "KC_LALT", "MO(1)", "LT(2,KC_SPC)", "KC_SPC", "MO(1)", "LT(3,KC_ENT)", "MO(5)", "KC_LEFT", "KC_DOWN", "KC_UP", "KC_RGHT"], ["KC_TRNS", "KC_F1", "KC_F2", "KC_F3", "KC_F4", "KC_F5", "KC_NO", "KC_TRNS", "KC_F6", "KC_F7", "KC_F8", "KC_F9", "KC_F10", "KC_F11", "KC_NO", "KC_EXLM", "KC_AT", "KC_HASH", "KC_DLR", "KC_P